<div style="background-color: #0070C0; height: 60px; color:white;padding: 0 0 0 0%; font-size:2rem; margin-right: 40%"> Laboratorio de generación de insights. </div>

El siguiente laboratorio contiene el código que se utilizó para crear las gráficas de la lección __insights__. Puedes jugar con el, cambiar el código y probar lo que te venga en mente. 

No es necesario que leas todo el código, pero __puedes aventurarte y contestar algunas preguntas que vienen en los comentarios__.

Conforme sigamos avanzando en el módulo vas a comprender porque la decisión de algunas partes del código.

# Outliers.

En esta sección mostramos el código detrás de un gráfico para ejemplificar cómo los outliers pueden ser insights interesantes a mostrar en un gráfico.

Primero importemos pandas, matplotlib y plotly

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as  px

Luego utilicemos el dataset de cars que viene en nuestro archivo zip

In [ ]:
df = pd.read_csv("../datasets/raw/cars.csv")
df

Para explorar e identificar algunos vehículos vamos a crear un gráfico con plotly

In [ ]:
import plotly.express as  px

px.scatter(x="weight", y="mpg",data_frame=df.reset_index(),hover_data=["index", "car name"])



Observa que __hay algunos outliers cerca a 3000 lb y entre 30 y 45 mpg__. Al pasar el mouse sobre esos puntos podemos extraer los índices y generar el gráfico de la celda siguiente.

In [ ]:
import matplotlib.pyplot as plt

#Configurando el tamaño del canvas------------------------------------------------------------
fig,ax = plt.subplots(1,figsize=(13.333,6.5)) #Configuración inicial del gráfico
fig.patch.set_facecolor('xkcd:white')         #Color del fondo de la gráfica

#Graficar todos los puntos --------------------------------------------------------------------
df.plot.scatter("weight",                     #Variable en x
                "mpg",                        #Variable en y
                ax=ax,                        #Objeto de matplotlib que contiene el gráfico
                s=90,                         #Tamaño de la geometría.
                c="#cccccc")                  #Color de la geometría

#Graficar individualmente: --------------------------------------------------------------------
#Graficar un punto
(df                                           #Dirigite al dataset "df"
  .iloc[387]                                  #Filtra por el índice 387
  .to_frame()                                 #Convierte el pandas series a dataframe
  .T                                          #Convierte el dataset en su transpuesta
  .plot                                       #Preparate a graficar
  .scatter("weight",                          #Variable en x
           "mpg",                             #Variable en y
           ax=ax,                             #Objeto de matplotlib que contiene el gráfico
           s=90,                              #Tamaño de la geometría.
           c="#4464c2"))                      #Color de la geometría de ese punto esécífico

#Graficar una anotación
ax.text(s = df.iloc[387]["car name"],         #Filtra por el indice 87 e imprime su "car name"
        x = df.iloc[387]["weight"] + 50,      #Filtra por el indice 87 y asigna variable x y mueve 50 unidades a la derecha"
        y = df.iloc[387]["mpg"],              #Filtra por el indice 87 y asigna variable y"
        size = 14)                            #Tamaño del texto

#Repetir paso anterior con puntos 327 y 333:
df.iloc[327].to_frame().T.plot.scatter("weight","mpg",ax=ax,s=90,c="#4464c2")
ax.text(s = df.iloc[327]["car name"], 
        x = df.iloc[327]["weight"] + 50, 
        y = df.iloc[327]["mpg"] - 1, 
        size = 14)


df.iloc[333].to_frame().T.plot.scatter("weight","mpg",ax=ax,s=90,c="#4464c2")
ax.text(s = df.iloc[333]["car name"], 
        x = df.iloc[333]["weight"] + 50, 
        y = df.iloc[333]["mpg"] - 0.5, 
        size = 14)

#Formato de componentes de un gráfico -----------------------------------------------------------
#Eje x
ax.set_xlabel("Peso (lb)",                     #Etiqueta del eje x
                size=20,                       #Tamaño de fuente
                loc="left",                    #Orientación a la izquierda
                color="#808080")               #Color de fuente
ax.tick_params(axis='x',                       #Configurar eje x
                labelsize=15,                  #Tamaños del texto de los ticks del eje
                labelcolor="#808080",          #Color del texto de los ticks del eje
                color="#808080")               #Color de los ticks

#Eje y
ax.set_ylabel("Millas por galón", size=20, loc="top", color="#808080")
ax.tick_params(axis='y', labelsize=15, color="#808080", labelcolor="#808080")

# Remover ejes sobrantes
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
#Cambiando el color de los ejes que se muestran
ax.spines['bottom'].set_color('#808080')
ax.spines['left'].set_color('#808080') 

# Mostrar solo ticks en los ejes que se grafican
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.set_yticks([0,10,20,30,40,50])              #Forzar al eje y a tener ticks específicos

ax.set_title('Algunos vehículos de peso "mediano" con alto rendimiento.\n',
             loc="left",                       #Colorcar título a la izquierda
             size=24)                          #Tamaño de fuente de 24pts

plt.show()


# Patrones diferentes.

El dataset sobre defunciones muestra diversas causas de defunción y su evolución a través del tiempo. La gran mayoría de las causas de defunción parecieran ir en incremento, siendo la principal las enfermedades cardiovasculares.

Sin embargo, existen países dónde esa tendencia incremental no se hace presente y, en cambio, tienen una tendencia a disminuir año año.

Ese es un ejemplo de otro tipo de insight, un patrón que se comporta diferente al resto.

Vamos a ver cómo resaltar ese patrón.

In [ ]:
#Cargando el dataset
df = pd.read_csv("../datasets/raw/20222703 Causes Of Death Clean Output V2.0.csv")

df.head(10)

Aqui vamos a utilizar lo que hemos visto de _data wrangling_.

Vamos a crear un método en cadena de pandas (_pipeline_). 

El primer paso es seleccionar y filtrar los datos por causa de defunción, __vamos a seleecionar las enfermedades cardiovasculres__. El segundo paso es __agrupar por año y país__, el agregado es la __suma del número de decesos.__

In [ ]:
df_cardio = (df[df["Causes name"]=="Cardiovascular diseases"]               #Filtrar por enfermedades cardiovasculares
                .groupby(["Year","Entity"],as_index=False)["Death Numbers"] #Agrupar y agregar
                .sum())

df_cardio

En el registro del dataset aparece `America` como una entidad, vamos a eliminarla del dataset y sobre escribir el nombre de `df_cardio`

In [ ]:
df_cardio = df_cardio[df_cardio["Entity"] != "America"]

__¿Qué países han disminuido los fallecimientos por enfermedades cardiovasculares?__ Para contestar esa pregunta podemos hacer una aproximación rápida y tomar sólo el año inicial (1990) y el año final (2019). Posteriormente __pivoteamos__ el dataset utilizando `Entity` como pivote (índice); luego __restamos__ el número de fallecimientos y nos quedamos con los países que tengan una pendiente negativa (disminución). 

In [ ]:
df2= (df_cardio[df_cardio["Year"].isin([1990,2019])]     #Filtra valores de 1990 y 2019 (¿qué es 'isin'?)
        .pivot_table(index="Entity",                     #Pivotea y asigna 'Entity' cómo index
                     columns="Year",                     #Las columnas serán los años
                     values="Death Numbers"))            #Los valores el número de decesos.

df2

In [ ]:
#Restar valores iniciales y finales
df2["slope"] = df2[2019] - df2[1990]

#Seleccionar aquellos con pendiente negativa:
df2[df2["slope"] <= 0]

Con el dataset `df2` podríamos saber que país tuvo una disminución significativa en los casos de enfermedades cardiovasculares.

In [ ]:
#Seleccionar aquellos con pendiente negativa y ordernar:
df2[df2["slope"] < 0].sort_values(by="slope")      #¿Qué hace sort_values?

Ahora sabemos que el país que queremos resaltar cómo Inisght es Reino Unido.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

#Configurando el tamaño del canvas------------------------------------------------------------
fig,ax = plt.subplots(1,figsize=(6.5,6.5))   #Configuración inicial del gráfico
fig.patch.set_facecolor('xkcd:white')         #Color del fondo de la gráfica

countries = df_cardio.Entity.unique()         #Generar una lista de países

for country in countries:                     #Se graficará país por país. Esto tomará ALGUNOS SEGUNDOS
    subset = df_cardio.loc[                   #.loc? que será el .loc?
                df_cardio["Entity"]==country] #Filtrar por país
    if country == "United Kingdom":           #Si el país es United Kingdom
        color="#4464c2"                       #Asigna un color azul
        z_index = 10                          #Colocar al frente

    else:                                     #Si no lo es:
        color="#cccccc"                       #Asigna un color gris
        z_index = 1                           #Colocar al fondo
    
    ax.plot(subset["Year"],                   #Asigna Year como eje x
            subset["Death Numbers"],          #Número de fallecimientos como Y
            c=color,
            zorder = z_index)                 #Y el color que seleccionamos en el paso anterior


#Crear dos subsets uno para 1990 y otro para 2019
subset90 = df_cardio.loc[df_cardio["Year"] == 1990]
subset19 = df_cardio.loc[df_cardio["Year"] == 2019]

#Agregar las anotaciones de países de contexto: --------------------------------------------------
countries = ["China", "India"]

#¿Puedes describir que hacen las líneas de código 35 q 46?
for country in countries:
    ax.text(s=country,
            x=2020,
            y=subset19[subset19["Entity"]==country]["Death Numbers"],
            c = "#8f8f8f",
            fontsize=12)
    ax.scatter(x = 1990,
               y = subset90[subset90["Entity"]==country]["Death Numbers"],
               c = "#c9c9c9")
    ax.scatter(x = 2019,
               y = subset19[subset19["Entity"]==country]["Death Numbers"],
               c = "#c9c9c9")
#-------------------------------------------------------------------------------------------------

#Agregar las anotaciones de países de insights:
#¿Puedes describir que hacen las siguientes líneas de código?
ax.text(s="Rusia reporta disminución de casos\nposterior al 2005",
        x=2020,
        y=subset19[subset19["Entity"]=="Russia"]["Death Numbers"],
        c= "#8f8f8f",
        fontsize=12)
ax.scatter(x = 1990,
            y = subset90[subset90["Entity"]=="Russia"]["Death Numbers"],
            c = "#c9c9c9")
ax.scatter(x = 2019,
            y = subset19[subset19["Entity"]=="Russia"]["Death Numbers"],
            c = "#c9c9c9")


ax.text(s="Reino Unido",
        x=2020,
        y=subset19[subset19["Entity"]=="United Kingdom"]["Death Numbers"],
        c= "#4464c2",
        fontsize=12)
ax.scatter(x = 1990,
            y = subset90[subset90["Entity"]=="United Kingdom"]["Death Numbers"],
            c = "#4464c2")
ax.scatter(x = 2019,
            y = subset19[subset19["Entity"]=="United Kingdom"]["Death Numbers"],
            c = "#4464c2")        

#------------------------------------------------------------------------------------------------
#Formato de componentes de un gráfico -----------------------------------------------------------
#Eje x
ax.set_xlabel("Años",                         #Etiqueta del eje x
                size=16,                      #Tamaño de fuente
                loc="left",                   #Orientación a la izquierda
                color="#808080")              #Color de fuente
ax.tick_params(axis='x',                      #Configurar eje x
                labelsize=12,                 #Tamaños del texto de los ticks del eje
                labelcolor="#808080",         #Color del texto de los ticks del eje
                color="#808080")              #Color de los ticks

#Eje y
ax.set_ylabel("Número de fallecimientos", size=16, loc="top", color="#808080")
ax.tick_params(axis='y', labelsize=12, color="#808080", labelcolor="#808080")

# Remover ejes sobrantes
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
#Cambiando el color de los ejes que se muestran
ax.spines['bottom'].set_color('#808080')
ax.spines['left'].set_color('#808080') 


# Mostrar solo ticks en los ejes que se grafican
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')


ax.set_title('Reino Unido ha logrado mantener a la\nbaja los fallecimientos por enfermedades \ncardiovasculares.\n',
             loc="left",                       #Colorcar título a la izquierda
             size=18)                          #Tamaño de fuente de 18pts

ax.yaxis.set_major_formatter(ticker.EngFormatter()) #Haciendo el formato del eje legible
ax.set_xlim(1988,2080)                              #Estableciendo un límite
                                                    #¿Qué pasa si cambias ese límite?
                                                    #¿Cambia la forma en que se ve el gráfico?
ax.set_xticks([1990,2005,2020])

plt.show()


# Explicar modelos de machine learning.

¿Recuerdas el módulo 3? En ese módulo abordamos la __importancia de variables__ y saber que variables te pueden funcionar en un modelo de machine learning también puede ser un _inisght_ importante.

Por ejemplo, basándonos en el notebook [07 - Importancia de Variables], podríamos obtener el siguiente gráfico y resaltar las variables con un coeficiente de correlación mayor a 0.5.

Al igual que en aquel notebook, carguemos nuestro dataset


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Shreyas3108/house-price-prediction/master/kc_house_data.csv')

df

Ahora, calculemos el coeficiente de correlación y seleccionemos únicamente la variable `price`.

Observa que hemos quitado la variable `id` y el precio después de crear el subset.

In [ ]:
price_df = (df.drop(columns=["id"])    #Remover columna id
            .corr()["price"]           #Mostrar precio
            .to_frame()                #Convertir a frame
            .sort_values(by="price")   #Ordenar valores
            .drop("price")             #Quitar price porque es la misma variable y R=1
            )            

price_df

Ahora podemos graficar el coeficiente de correlación de nuestras variables.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

#Configurando el tamaño del canvas------------------------------------------------------------
fig,ax = plt.subplots(1,figsize=(13.333,6.5))          #Configuración inicial del gráfico
fig.patch.set_facecolor('xkcd:white')                  #Color del fondo de la gráfica
   
price_df.plot.barh(ax=ax,legend=False)                 #Al utilizar pandas como 'render' genera leyendas
                                                       #de manera automatica, por eso las forzamos
                                                       #a que sea False

#Cambiar color sólo si R >0.5
prices = price_df["price"].to_list()                   #Vamos a iterar sobre todas las variables
for i,_ in enumerate(prices):
    if prices[i] > 0.5:                                #Si R es mayor a 0.5 entonces:
        ax.get_children()[i].set_color("#4464c2")      #Colorea de azul el elemento i
    else:
        ax.get_children()[i].set_color("#c9c9c9")      #Sino, colorea ese elemento i de gris
    
    #Agregar los valores porque vamos a quitar el eje x
    ax.text(s = str(np.round(prices[i],2)),            #Aprovechamos el for loop para agregar los valores
            x = prices[i]+.01,
            y = i-.2,
            color="#808080")

#------------------------------------------------------------------------------------------------
#Formato de componentes de un gráfico -----------------------------------------------------------
#Eje y
ax.tick_params(axis='y', labelsize=14, color="#808080", labelcolor="#808080")

# Remover todos los ejes y dejar sólo las variables.
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.xaxis.set_ticklabels([])
ax.xaxis.set_ticks([])

ax.set_title('Coeficiente de correlación para los predictores de la variable precio.\n',
             loc="right",                       #Colorcar título a la izquierda
             size=24)                          #Tamaño de fuente de 18pts

plt.show()

# Fin del laboratorio.

Tal como pudiste ver, hay manera de resaltar los insights que consideremos importantes. En este notebook te mostramos como crear gráficos orientados a resaltar un aspecto importante de la información contenida en un gráfico.

Al final tu podrías resaltar el patrón o información que creas más importante o interesante para compartir. Recuerda, eso dependerá del proyecto y lo que estes buscando resolver con tus datos.

En las lecciones siguientes profundizaremos en otros conceptos que te permitirán apreciar porque es importante ir conociendo esta forma de manipular tus gráficos.